In [1]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hye085').getOrCreate()

In [29]:
df = spark.read.csv('Datasets/hotel_bookings.csv',inferSchema=True,header=True)

In [4]:
df.columns

['hotel',
 'is_canceled',
 'lead_time',
 'arrival_date_year',
 'arrival_date_month',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'reserved_room_type',
 'assigned_room_type',
 'booking_changes',
 'deposit_type',
 'agent',
 'company',
 'days_in_waiting_list',
 'customer_type',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'reservation_status',
 'reservation_status_date']

In [7]:
df.head()

Row(hotel='Resort Hotel', is_canceled=0, lead_time=342, arrival_date_year=2015, arrival_date_month='July', arrival_date_week_number=27, arrival_date_day_of_month=1, stays_in_weekend_nights=0, stays_in_week_nights=0, adults=2, children='0', babies=0, meal='BB', country='PRT', market_segment='Direct', distribution_channel='Direct', is_repeated_guest=0, previous_cancellations=0, previous_bookings_not_canceled=0, reserved_room_type='C', assigned_room_type='C', booking_changes=3, deposit_type='No Deposit', agent='NULL', company='NULL', days_in_waiting_list=0, customer_type='Transient', adr=0.0, required_car_parking_spaces=0, total_of_special_requests=0, reservation_status='Check-Out', reservation_status_date=datetime.datetime(2015, 7, 1, 0, 0))

In [10]:
df.printSchema()

root
 |-- hotel: string (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: integer (nullable = true)
 |-- arrival_date_day_of_month: integer (nullable = true)
 |-- stays_in_weekend_nights: integer (nullable = true)
 |-- stays_in_week_nights: integer (nullable = true)
 |-- adults: integer (nullable = true)
 |-- children: string (nullable = true)
 |-- babies: integer (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullab

In [11]:
df.select("hotel").show()

+------------+
|       hotel|
+------------+
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
|Resort Hotel|
+------------+
only showing top 20 rows



In [14]:
df.count()

119390

In [30]:
import pyspark.sql.functions as pyfunc
from functools import reduce
df_agg_col = reduce(
    lambda a, b: a.union(b),
    (
        df.agg(
            pyfunc.count(pyfunc.when(pyfunc.isnull(c), c)).alias('NULL_Count')
        ).select(pyfunc.lit(c).alias("Column_Name"), "NULL_Count")
        for c in df.columns
    )
)
df_agg_col.show(len(df.columns))

+--------------------+----------+
|         Column_Name|NULL_Count|
+--------------------+----------+
|               hotel|         0|
|         is_canceled|         0|
|           lead_time|         0|
|   arrival_date_year|         0|
|  arrival_date_month|         0|
|arrival_date_week...|         0|
|arrival_date_day_...|         0|
|stays_in_weekend_...|         0|
|stays_in_week_nights|         0|
|              adults|         0|
|            children|         0|
|              babies|         0|
|                meal|         0|
|             country|         0|
|      market_segment|         0|
|distribution_channel|         0|
|   is_repeated_guest|         0|
|previous_cancella...|         0|
|previous_bookings...|         0|
|  reserved_room_type|         0|
|  assigned_room_type|         0|
|     booking_changes|         0|
|        deposit_type|         0|
|               agent|         0|
|             company|         0|
|days_in_waiting_list|         0|
|       custom

In [28]:
nan_replacements = {"children:": 0.0,"country": "Unknown", "agent": 0, "company": 0}
full_data_cln = full_data.fillna(nan_replacements)